In [1]:
import numpy as np
import scipy
from scipy.io import loadmat
import mne, glob
from mne_features.feature_extraction import extract_features
import pandas as pd
from ipynb.fs.full.fullDataExtraction import getRawArrayData
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go
import plotly
import math

In [2]:
channels=pd.read_excel("Preprocessed_EEG/channel-order(viz).xlsx")
channelNames=channels.iloc[:,0]
channelNames=np.ndarray.tolist(pd.Series.to_numpy(channelNames))
channelNames.insert(0,"Fp1")
info=mne.create_info(channelNames,200,'eeg')
subject=loadmat('1_20131107.mat')
ch_types = ['eeg'] * len(channelNames)
info = mne.create_info(channelNames, ch_types=ch_types, sfreq=200)
info.set_montage('standard_1020')
data=np.delete(subject['djc_eeg1'], 61, 0)
data=np.delete(data, 57, 0)
evoked=mne.EvokedArray(data,info)
transposedData=np.transpose(data)
df = pd.DataFrame(transposedData, columns = channelNames)
df = df.head(500)

In [3]:
def makeValueMatrix(currentAnalysed):
    result=pd.read_excel("Preprocessed_EEG/channel-order(topo).xlsx").to_numpy().astype(np.float32)
    #tracker value
    k=0
    for i in range(0,result.shape[0]):
        for j in range (0,result.shape[1]):
            if not(math.isnan(result[i][j])):
                result[i][j]=currentAnalysed[k]
                k+=1
    
    return result

In [4]:
def makePostionList(data):
    position=[]
    for column in range (0,500):
        position.append(makeValueMatrix(data[:,column]))
    return position

In [5]:
lineChartInstances=[]
for row in range(df.shape[0]):
    lineChartInstances.append(go.Scatter(y=df.head(row)))

In [6]:
points=makePostionList(data)

heatMapInstances=[]
for data in points:
    heatMapInstances.append(go.Heatmap(z=data,coloraxis = "coloraxis"))

from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=1)
fig=fig.add_traces(heatMapInstances,rows=1,cols=1)

steps = []
for i in range(500):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(fig.data)],
        name = i
    )
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(steps = steps,)]

In [7]:
fig.layout.sliders = sliders

In [8]:
go.FigureWidget(fig)


FigureWidget({
    'data': [{'coloraxis': 'coloraxis',
              'type': 'heatmap',
              'uid': '…

--------------

In [6]:
frames = [
    go.Frame(data=go.Heatmap(z=points[i]),name=i)
    for i in range(0,500)
]

In [7]:
slider=[{"steps": [{"args": [[f.name],{"frame": {"duration": 0, "redraw": True},
                                            "mode": "immediate",},],
                         "label": f.name, "method": "animate",}
                        for f in frames],}]

In [8]:
fig=go.Figure(data=frames[0].data, frames=frames).update_layout(
    updatemenus=[
        {
            "buttons": [{"args": [None, {"frame": {"duration": 500, "redraw": True}}],
                         "label": "Play", "method": "animate",},
                        {"args": [[None],{"frame": {"duration": 0, "redraw": False},
                                          "mode": "immediate", "transition": {"duration": 0},},],
                         "label": "Pause", "method": "animate",},],
            "type": "buttons",
        }
    ],
    sliders=slider,
    height=800,
)